In [1]:
import os
os.chdir('..')

In [2]:
import cudf

In [23]:
clean_cdr = cudf.read_csv('data/processed/201308/clean_cdr.csv', dtype={'calling_area_code': int, 'called_area_code': int})
meta_tower = cudf.read_csv('data/processed/meta_tower.csv')

meta_tower['cell_id'] = meta_tower['CELL_16'].copy()
meta_tower

,CELL_10,CELL_16,LON,LAT,X,Y,DEYANG_CENTER,call_area_code,cell_id
0,1,0001,103.993,30.7214,403.546,3400.52,0,28,0001
1,2,0002,103.993,30.7214,403.546,3400.52,0,28,0002
2,3,0003,103.993,30.7214,403.546,3400.52,0,28,0003
3,4,0004,104.161,30.6291,419.562,3390.16,0,28,0004
4,5,0005,104.161,30.6291,419.562,3390.16,0,28,0005
...,...,...,...,...,...,...,...,...,...
40564,65512,FFE8,103.966,30.5730,400.809,3384.09,0,28,FFE8
40565,65520,FFF0,103.980,30.6016,402.180,3387.25,0,28,FFF0
40566,65522,FFF2,103.966,30.5730,400.809,3384.09,0,28,FFF2
40567,65524,FFF4,104.066,30.5781,410.407,3384.57,0,28,FFF4


In [24]:
N = len(clean_cdr)
clean_cdr

,serv_id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id
0,9afmdpyv,ejra2suz,1,5,ejra2suz,838,53,838,4ofzl4yr,3CC3
1,9afmdpyv,ejra2suz,1,5,ejra2suz,838,27,838,4ofzl4yr,3CC3
2,9aqm8dgb,gh8w8ki0,1,5,9pzs3exd,838,61,838,gh8w8ki0,351D
3,a2fug2rm,ftik422s,1,5,ftik422s,838,53,838,931s2hft,36B8
4,a2fug2rm,ftik422s,1,5,ftik422s,838,45,838,931s2hft,36B7
...,...,...,...,...,...,...,...,...,...,...
52978352,8rzedibz,66ak2s5v,17,7,66ak2s5v,838,62,838,moyl2k57,3649
52978353,ahn6i8mj,ltzkksuv,17,7,ltzkksuv,838,148,838,hjo0ksut,3B56
52978354,8vb94qft,njo45k8v,17,7,8yro82d5,28,46,838,njo45k8v,394C
52978355,a5aijq0l,9ejejxlp,17,7,jeg1c6wi,28,35,838,9ejejxlp,153F


In [25]:
group1 = clean_cdr[
    (clean_cdr['calling_area_code'] == 838) &
    (clean_cdr['called_area_code'] != 838)
].copy()
n1 = len(group1)

group2 = clean_cdr[
    (clean_cdr['calling_area_code'] != 838) &
    (clean_cdr['called_area_code'] == 838)
].copy()
n2 = len(group2)

print(f'total number oberservations in cdr: {N}\n')


n_ = len(clean_cdr[
    (clean_cdr['calling_area_code'] == 838) &
    (clean_cdr['called_area_code'] == 838)
])
print(f'total number of observations which both calling and called are local: {n_}')
print(f'percentage: {n_ / N * 100}%\n')

print(f'total number of observation in group1: {n1}')
print(f'percentage: {n1 / N * 100}%\n')

print(f'total number of observation in group2: {n2}')
print(f'percentage: {n2 / N * 100}%')

total number oberservations in cdr: 52978357

total number of observations which both calling and called are local: 44410897
percentage: 83.82837731264486%

total number of observation in group1: 4492391
percentage: 8.479672180094223%

total number of observation in group2: 3937129
percentage: 7.431580031823184%


# Client number?

In [42]:
print('percentage of observations in cdr which client_nbr is calling_nbr: ')
print(f"{(clean_cdr['client_nbr'] == clean_cdr['calling_nbr']).sum() / N * 100}%")

percentage of observations in cdr which client_nbr is calling_nbr: 
53.48110172612564%


In [45]:
print('percentage of observations in cdr which client_nbr is called_nbr: ')
print(f"{(clean_cdr['client_nbr'] == clean_cdr['called_nbr']).sum() / N * 100}%")

percentage of observations in cdr which client_nbr is called_nbr: 
46.51912289390175%


In [43]:
print('percentage of observations in group1 which client_nbr is calling_nbr: ')
print(f"{(group1['client_nbr'] == group1['calling_nbr']).sum() / n1 * 100}%")

percentage of observations in group1 which client_nbr is calling_nbr: 
100.0%


In [44]:
print('percentage of observations in group2 which client_nbr is called_nbr: ')
print(f"{(group2['client_nbr'] == group2['called_nbr']).sum() / n2 * 100}%")

percentage of observations in group2 which client_nbr is called_nbr: 
100.0%


# Experiment - group1

In [26]:
group1_ = cudf.merge(group1, meta_tower, on='cell_id')

group1_

,serv_id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id,CELL_10,CELL_16,LON,LAT,X,Y,DEYANG_CENTER,call_area_code
0,9u6k7puf,8nhdjzzt,12,5,8nhdjzzt,838,42,816,2n9sk2oy,2B31,11057,2B31,104.740,31.4594,475.289,3481.94,0,816
1,8urfd3fq,hytw3mkh,12,5,hytw3mkh,838,99,391,ljpvloyf,37D5,14293,37D5,104.395,31.1036,442.282,3442.62,0,838
2,9f1zg2ug,8bis5s6s,12,5,8bis5s6s,838,110,23,gdeu5mmw,3C9F,15519,3C9F,104.204,31.3509,424.257,3470.15,0,838
3,9f1zg2ug,8bis5s6s,12,5,8bis5s6s,838,29,23,gdeu5mmw,3C9F,15519,3C9F,104.204,31.3509,424.257,3470.15,0,838
4,8ru8l8kn,9rgj3aol,12,5,9rgj3aol,838,355,734,lu62anyr,3471,13425,3471,104.416,31.2405,444.366,3457.79,0,838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309618,9pmp3fog,8jel2l54,12,7,8jel2l54,838,106,21,n8bcf0wr,3778,14200,3778,104.689,31.0659,470.319,3438.32,0,838
4309619,a4zkaya0,9pq83alt,12,7,9pq83alt,838,61,28,dgy165uj,078F,1935,078F,104.074,30.6313,411.223,3390.47,0,28
4309620,aknjdqcs,gkn43aoo,12,7,gkn43aoo,838,351,817,hp14jzm1,0D3A,3386,0D3A,103.978,30.8281,402.217,3412.36,0,28
4309621,ap9saaic,2c384hnq,12,7,2c384hnq,838,40,835,fjhbche8,38C1,14529,38C1,104.381,31.1133,440.952,3443.70,1,838


In [30]:
print('case when cell location is the same with calling:')
n1_ = len(group1_[group1_['calling_area_code'] == group1_['call_area_code']])
print(n1_)
print(f'{n1_ / 4309623 * 100}%')

case when cell location is the same with calling:
3094028
71.79347242206569%


In [31]:
print('case when cell location is the same with called:')
n2_ = len(group1_[group1_['called_area_code'] == group1_['call_area_code']])
print(n2_)
print(f'{n2_ / 4309623 * 100}%')

case when cell location is the same with called:
892070
20.699490419463604%


In [33]:
print('remained:')
print(f'{(1 - n1_ / 4309623 - n2_ / 4309623) * 100}%')

remained:
7.507037158470712%


# Experiment - group2

In [34]:
group2_ = cudf.merge(group2, meta_tower, on='cell_id')

group2_

,serv_id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id,CELL_10,CELL_16,LON,LAT,X,Y,DEYANG_CENTER,call_area_code
0,9ngj8eax,mwtx2kgb,10,5,3i0okpdm,28,21,838,mwtx2kgb,35E3,13795,35E3,104.775,30.7572,478.457,3404.08,0,838
1,9htwkdmo,mgpc5zwr,10,5,fa933i0k,817,40,838,mgpc5zwr,D16A,53610,D16A,106.060,30.8222,601.425,3411.74,0,817
2,a2l7jb78,4h912kiv,10,5,2fvxkzve,28,103,838,4h912kiv,3ADC,15068,3ADC,104.374,31.0913,440.271,3441.27,1,838
3,9fg4cvkj,kzdx2kaq,10,5,bju0l51m,28,236,838,kzdx2kaq,377A,14202,377A,104.369,31.1019,439.800,3442.44,0,838
4,8zum5dfg,d8xhjhh7,10,5,filx6ymo,23,347,838,d8xhjhh7,3C8F,15503,3C8F,104.407,31.1251,443.440,3445.00,1,838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781632,8v5u2lct,kjyt387u,16,7,goucj4wg,979,404,838,kjyt387u,34A7,13479,34A7,104.558,31.3948,457.962,3474.83,0,838
3781633,9pv9f8fx,brdom2oq,17,7,ieey3uqx,813,5,838,brdom2oq,9B42,39746,9B42,104.758,29.3398,476.497,3246.96,0,813
3781634,a9rredcl,mdiu286n,17,7,kwxc3pvy,816,165,838,mdiu286n,3A93,14995,3A93,104.382,31.1090,441.045,3443.23,1,838
3781635,9pv9f8fx,brdom2oq,17,7,ieey3uqx,813,84,838,brdom2oq,9A56,39510,9A56,104.760,29.3337,476.690,3246.28,0,813


In [35]:
print('case when cell location is the same with calling:')
n1_ = len(group2_[group2_['calling_area_code'] == group2_['call_area_code']])
print(n1_)
print(f'{n1_ / 3781637 * 100}%')

case when cell location is the same with calling:
715058
18.90868954370819%


In [36]:
print('case when cell location is the same with called:')
n2_ = len(group2_[group2_['called_area_code'] == group2_['call_area_code']])
print(n2_)
print(f'{n2_ / 3781637 * 100}%')

case when cell location is the same with called:
2768451
73.20774045737335%


In [37]:
print('remained:')
print(f'{(1 - n1_ / 3781637 - n2_ / 3781637) * 100}%')

remained:
7.88356999891846%
